# Filtrage collaboratif

*ismael Bonneau*

Le filtrage collaboratif (en anglais: collaborative filtering) est une méthode utilisée par les systèmes de recommandations.

Une méthode de recommandation classique et de recommander à un utilisateur des objets similaires de celui qu'il a déjà aimé. Cette similarité, dans le cas d'une série ou d'un film par exemple peut se baser sur le genre, les acteurs en commun, le synopsis... Ainsi, si un utilisateur a aimé la série the punisher et luke cage, le système lui recommandera daredevil, shield agents, etc...
Cette stratégie a un défaut: les recommandations manquent de diversité et n'incitent pas l'utilisateur à explorer le catalogue.

Une deuxième approche est le **filtrage collaboratif**:
il permet de réaliser des prédictions automatiques ("filtrage") des intérêts d'un utilisateur en se basant sur les préférences d'un grand nombre d'autres utilisateurs ("collaboratif"), afin de recommander des produits (films, séries, musique, articles sur un site de e-commerce...) pertinents pour un utilisateur.

<img src="images/filtragecollaboratif.png" width="600" />

-------------------------------------------------------------------------

### Principe:

L'hypothèse sous-jacente du filtrage collaboratif est que si une personne A a la même opinion qu'une personne B sur un sujet, A a plus de chance d'avoir la même opinion que B sur un autre sujet qu'une personne choisie au hasard.

Le système commence donc d'abord par collecter des avis d'un grand nombre d'utilisateurs sur un grand nombre d'objets (dans notre cas, des séries). Cet avis peut prendre plusieurs formes (1-5 étoiles, note sur 10, j'aime/je n'aime pas...) 

Puis, pour un utilisateur A le système trouve les utilisateurs qui ont les goûts les plus similaires. A partir des goûts de ces utilisateurs les plus similaires, le système peut prédir à l'utilisateur A une note pour chacun des objets qu'il n'a pas noté. 

Plusieurs types d'approche existent:

1) l'approche dite **memory-based**:
<p>
    Cette approche utilise les notes attribuées par les utilisateurs pour calculer la similarité entre les utilisateurs ou les objets. Elle se base sur un calcul de similarité et utilise des algorithmes classiques comme:
    <ul>
        <li>K plus proches voisins (K-NN) <a href="https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm">wikipédia</a></li>
        <li>Des mesures de similarité comme la similarité cosinus, la corrélation de Pearson...
            <a href=""></a></li>
    </ul>
</p>

2) l'approche dite **model-based**: 
<p>
    Cette approche utilise des techniques de machine learning et de data mining pour attribuer des notes couples utilisateur-objet. 
<ul>
    <li>Décomposition en valeurs singulières (SVD) <a href="https://en.wikipedia.org/wiki/Singular_value_decomposition">wikipédia</a></li>
    <li>Factorisation de matrice non négative (NNMF) <a href="https://en.wikipedia.org/wiki/Non-negative_matrix_factorization">wikipédia</a></li>
    <li>Bayesian Personalized Ranking (n'attribue pas de "notes" mais un classement) <a href="https://cran.r-project.org/web/packages/rrecsys/vignettes/b6_BPR.html">lien</a></li>
    <li>...Et bien d'autres (approches à base de clustering...)</li>
</ul>
</p>

pour en savoir plus sur le filtrage collaboratif: <a href="https://en.wikipedia.org/wiki/Collaborative_filtering">wikipédia (en anglais)</a>

<img src="images/Classification-of-collaborative-filtering-algorithms.png" width="600" />

image sources:

<a href="https://www.researchgate.net/profile/Kan_Zheng/publication/303556519/figure/fig4/AS:614297214414873@1523471277992/Classification-of-collaborative-filtering-algorithms.png">[1]</a> <a href="https://johnolamendy.wordpress.com/2015/10/14/collaborative-filtering-in-apache-spark/">[2]</a>

-------------------

### Notre but:

Nous allons mettre en oeuvre et comparer plusieurs approches de recommandation collaborative, en l'occurence les approches model-based. 

Notre but est d'implémenter et comparer qualitativement et quantitativement les algorithmes de factorisation de matrice non négative (NNMF), décomposition en valeurs singulières SVD, éventuellement Bayesian Personalized Ranking, et différentes fonctions de coûts associées, sur un jeu de données collectés sur le site imdb.

Notre algorithme final sera hybride, c'est à dire qu'il intègrera une mesure de similarité item, pour améliorer la qualité de la recommandation.

### Données:

Nous partons d'une base de ${m = 48705}$ utilisateurs ayant noté ${n = 892}$ séries. Ces données sont extraites du site <a href="https://www.imdb.com/">imdb</a> (voir script <a href="https://github.com/ismaelbonneau/movie_recommender/blob/master/scraping/scrap.py">scraping/scraping.py</a>) et sont résumées dans une matrice de taille ${n,m}$ où chaque entrée ${(u, i)}$ de matrice contient la note que l'utilisateur ${u}$ a attribué à l'item (série) ${i}$, sur 10 (le site ayant choisi un système de notation sur 10 étoiles).

---------------------

### Rentrons dans le vif du sujet:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import dok_matrix, csr_matrix #matrice "sparse"
import seaborn as sns

from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

sns.set(color_codes=True)
plt.style.use('seaborn-white')
%matplotlib inline

%load_ext autoreload
%autoreload 2

#dataset path
path = "dataset"

filename = "userratings.csv"
tableSeries = "series.csv"

### Récupérons les avis utilisateur

In [2]:
df = pd.read_csv(filename)
print("il y a {} utilisateurs et {} séries".format(df.shape[1], df.shape[0]))

il y a 48705 utilisateurs et 892 séries


**Ci dessous**, les premières lignes de la matrice, avec en ligne les séries et en colonne les utilisateurs. On peut voir que la plupart des cases sont vides (NaN), les utilisateurs n'ayant noté que très peu de séries.

In [3]:
df.head(4)

,Unnamed: 0,ToddTee,bkoganbing,betwana,fabiogaucho,killer1h,rasadi27,michael_cure,MashedA,drarthurwells,...,jimjohnson-57331,tatianavoloshka,hectorgarcia-41182,allisonbryan-30611,timothyquaid,eduardoellis,Chris_Tsimpoukas,ToxicAvox,DinoLord94,bratdawg
0,tt0035665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0042114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0043208,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0047708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Jetons un oeil à la distribution des données:

In [69]:
for seuil in [1, 5, 10]:
    c = len([a for a in (df.count(axis=0) > seuil) if a])/df.shape[1]
    print("{0:.2f}% des utilisateurs ont noté plus de ".format(
        100*c)+str(seuil)+" série ("+str(int(c*df.shape[1]))+" total)")
print("\n")   
for seuil in [2, 20, 50, 100]:
    c = len([a for a in (df.count(axis=1) > seuil) if a])/df.shape[0]
    print("{0:.2f}% des series ont reçu plus de ".format(
        100*c)+str(seuil)+" notes ("+str(int(c*df.shape[0]))+" total)")

13.55% des utilisateurs ont noté plus de 1 série (6598 total)
1.38% des utilisateurs ont noté plus de 5 série (670 total)
0.46% des utilisateurs ont noté plus de 10 série (226 total)


98.54% des series ont reçu plus de 2 notes (879 total)
69.62% des series ont reçu plus de 20 notes (621 total)
35.87% des series ont reçu plus de 50 notes (320 total)
19.28% des series ont reçu plus de 100 notes (172 total)


<img src="images/distribution_avis.png" />

L'histogramme révèle que les utilisateurs donnent très peu d'avis: 86,5% d'entre eux n'ont noté qu'une série. Les séries ont quant à elle plus d'avis: près de 20% des séries ont reçu plus de 100 notes, et 70% des séries ont reçu au moins 20 notes. Il faut choisir un **seuil de coupure** pour éliminer les utilisateurs ayant trop peu noté: nous choisissons arbitrairement de couper en-dessous de 4. Pour les séries, on gardera celles ayant reçu au moins 4 notes.

<img src="images/distribution_notes.png" />

### Résultat:

Beaucoup de ${1/20}$, beaucoup de ${10/10}$. Cet histogramme illustre un **phénomène connu en recommandation**: les utilisateurs ont tendance à ne noter que les films qui les ont marqués, soit positivement (j'ai adoré, je donne ${10/10}$) soit négativement (j'ai détesté, je donne un ${1/10}$). Ce phénomène est particulièrement illustré sur les utilisateurs qui notent peu: les utilisateurs qui notent beaucoup, avec un comportement sur le site de "critique amateur" ont eu tendance à noter la plupart des films qu'ils voient, et donc à donner des notes plus variées.

Il existe un autre phénomène en recommandation: 
On dit que les données sont 
- **MCAR** (Missing Completely At Random, en français "manquantes complètement au hasard"). 
- **MAR** (Missing At Random, en français "manquantes au hasard"). Contrairement à ce que l'on peut penser 

Retirons les noms de série, et enlevons les utilisateurs ayant noté moins de 7 séries et les séries notées moins de 15 fois:

In [7]:
print("{0:.2f}% de la matrice remplie".format((df.count().sum() / df.values.size) * 100))

0.15% de la matrice remplie


0.15% de la matrice remplie seulement!!

In [8]:
series = pd.read_csv("series.csv")
series = series[["seriesname", "imdbId"]]

df.rename(columns={"Unnamed: 0":'item'}, inplace=True)
df = df.loc[:, (df.count(axis=0) >= 7)] #enlever les utilisateurs ayant noté moins de 5 séries
df = df.loc[(df.count(axis=1) >= 15), :] #enlever les séries ayant reçu moins de 4 notes
print("il y a {} utilisateurs et {} séries".format(df.shape[1], df.shape[0]))

df = df.join(series.set_index("imdbId"), on="item")

df.dropna(subset=["seriesname"],inplace=True)
df.drop_duplicates(subset='item', keep='first', inplace=True)

df = df.set_index("seriesname")
df.drop(["item"],axis=1, inplace=True)

il y a 516 utilisateurs et 171 séries


In [9]:
print("{0:.2f}% de la matrice remplie".format((df.count().sum() / df.values.size) * 100))

4.80% de la matrice remplie


In [10]:
df.shape

(171, 515)

In [ ]:
#dataframe préparé à l'avance pour gagner du temps
df = pd.read_csv("userratingstruncated.csv")

<img src="images/distribution_notes_apresfiltrage.png" />

Regardons les séries les mieux notées:

In [20]:
for s in np.argsort(np.array(df.mean(axis=1)))[::-1][:10]:
    print(" ".join(df.index[s].split("_")[1:]))

Avatar The Last Airbender
Only Fools and Horses
Six Feet Under
Father Ted
I Love Lucy
Twilight Zone
All in the Family
Doctor Who (1963)
The Wire
Curb Your Enthusiasm


### Masquage des valeurs manquantes:

Nous allons créer une variable "masque" booléen, qui nous servira pour l'étape suivante. On obtient un tableau de même dimension, dans lequel False indique une valeur manquante (à masquer pour la suite) et True une valeur observée.

### Séparons nos données observées en train/test:

On sépare nos données en ensemble train/test. 40% de données observées en test, 60% en train.

In [114]:
import random

train = df.notnull().values
test_size = 0.4
test = train.copy()

for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        if train[i,j]:
            if random.random() < test_size:
                #ajouter dans test et enlever de train
                train[i,j] = False
            else:
                test[i,j] = False

### Matrix Factorization: principe

Une des façons de faire du filtrage collaboratif par approche model-based est d'utiliser un algorithme de **factorisation de matrice**. 

On pose ${\mathbf{R}}$ la matrice des notes, $\mathbf{\hat{R}}$ la matrice que l'on cherche à construire, contenant toutes les notes existantes et prédites. Cette matrice est de dimension ${m, n}$ avec ${m}$ le nombre d'utilisateurs et ${n}$ le nombre de séries. On va chercher à trouver deux matrices "facteur" $\mathbf{U}$ et $\mathbf{I}$ de dimension ${m, k}$ et ${k, n}$ de telle sorte que $\mathbf{\hat{R}}=\mathbf{U}\cdot\mathbf{I}$ avec ${\mathbf{\hat{R}} \approx \mathbf{R}}$

<img src="images/collaborative_filtering.png" width="400" />

Les dimensions ${m}$ et ${n}$ étant connues à l'avance, c'est la dimension ${k}$ qu'il reste à fixer, et définir une "fonction" mathématique qui mesure à quel point les deux matrice $\mathbf{R}$ (reconstituée) et la matrice originale sont proches.

Ainsi, et d'après la définition du produit matriciel, chaque case de la matrice $R_{i,j}$ résultante est le résultat d'une combinaison linéaire de $U_{i,}$ et $I_{,j}$, c'est à dire d'un vecteur de dimension ${k}$ représentant l'utilisateur ${i}$ et d'un vecteur de dimension ${k}$ représentant l'item ${j}$.

Il s'agit donc de projeter les utilisateurs et les items dans un espace de dimension ${k}$ ! On dit aussi qu'on apprend des _profils utilisateur_ et des _profils item_ sur ${k}$ variables latentes.

Plusieurs algorithmes peuvent servir à réaliser une factorisation de matrice.

### NMF (non negative matrix factorization): 

Un algorithme de factorisation de matrice simple. L'algorithme consiste en une descente de gradient sur une fonction d'erreur. L'algorithme ne tient pas compte des valeurs manquantes: Il effectue une descente de gradient uniquement sur les valeurs observées. Il "apprend" donc les relations entre items et utilisateurs sur les valeurs observées, pour ensuite pouvoir prédire les valeurs manquantes.

on calcule en fait ${\hat{R_{i,j}}= U_{i}\cdot I_{j}}$.

On apprend donc deux matrices ${U}$ et ${I}$. 
la fonction d'erreur est ${e = ||(R -\hat{R})||^2 + \beta (||U|| + ||I||)}$.

${\beta}$ est le paramètre de régularization. Il nécessite d'être ajusté. Il sert à limiter le nombre de valeurs non nulles dans ${U}$ et ${I}$, afin de s'assurer que l'on utilise que les variables latentes utiles. Ce type de régularisation est appellé régularisation L1. Pour en savoir plus sur la régularisation en machine learning: <a href="">lien</a> 

La particularité de l'algorithme est qu'il intègre une contrainte sur les valeurs des matrices ${U}$ et ${I}$: elles doivent être **positives** (d'où le "non negative").<br>
On a donc ${U_{i,k} \geq 0}$ ${\forall{(i,k)} \in \{1, ...,m\} \times \{1, ...,k\}}$ et ${I_{k,j} \geq 0 \forall{(k,j)} \in \{1, ...,k\} \times \{1, ...,n\}}$.

Cette contrainte de non-négativité, qui fait la particularité de NMF présente deux avantages: 1) le modèle obtenu est expliquable: puisque les affinités sont toutes additives, on comprend plus aisément comment les intérêts se combinent entre eux.


##### petit problème: données manquantes

Un petit problème se pose dans la partie applicative: la matrice ${R}$ contient des données manquantes qui sont, dans une matrice sparse, interprétés comme des 0. Ce ne sont pourtant pas des notes de 0 mais bien une **absence de note** (notez la différence). Les implémentations des algorithmes de factorisation de matrice de scikit-learn ne prennent pas ce point en compte et cherchent à prédire des 0 à la place des valeurs manquantes. On ne peut donc pas utiliser les implémentations de scikit-learn et il faut définir notre propre implémentation (ou utiliser gensim/surprise).

Ces valeurs manquantes sont en fait un vrai problème: en calculant l'erreur uniquement sur les valeurs observées, la fonction d'erreur devient non convexe et la descente de gradient a de très fortes chances de tomber dans un minimum local.

### Implémentation dans tensorflow:

voici un exemple. le code sous forme de classe se trouve dans <a href="https://github.com/ismaelbonneau/movie_recommender/tree/master/recommenders">le package recommenders</a>

In [24]:
from recommenders.matrixFactorization import NMF #implémentation sous forme de classe avec tensorflow

In [115]:
#moyenne item
baseline = np.tile(df.mean(axis=1).values, (df.shape[1],1)).transpose()

nmf = NMF(10)
learnt_U_nmf, learnt_I_nmf, results = nmf.run(df.transpose(), train.transpose(), test.transpose(), 1000, 0.00001, alpha=0.001, verbose=False)

In [68]:
recommandees = np.clip(np.dot(learnt_U_nmf, learnt_I_nmf), a_min=0, a_max=10)
print("dimensions de la matrice reconstituée: ", recommandees.shape)

dimensions de la matrice reconstituée:  (515, 171)


In [107]:
from utils.evalIRmodel import mse_mae

In [116]:
mse_mae(df, test, recommandees.T, baseline)

mse:  4.441516225353401
mae:  1.5091232859077266
mse baseline:  6.035569205546262
mae baseline:  1.9008648469406044


## Un peu d'hyperparameters tuning s'impose:

Une étpe cruciale maintenant, chercher les paramètre optimaux correspondants à notre modèle sur notre tâche.

Dans notre cas, les hyper paramètres sont:
- le ***nombre de facteurs (${k}$)***, qui peut favoriser le sur apprentissage,
- le coefficient **${\beta}$** qui est notre ***coefficient de régularization L1***,
- le ***nombre d'itérations***,
- le ***learning rate (taux d'apprentissage)***, qui règle la "vitesse de convergence",
- et...
- l'initialisation et la méthode d'optimisation, qui ne sont pas à proprement parler des hyperparamètres mais sur lesquels nous pouvons nous poser des questions et tester plusieurs variantes.

> Pour se comparer à quoi, par rapport à quoi?

La factorisation de matrices très sparse est un problème difficile très sujet au sur apprentissage. Nous séparons nos données observées (les données connues) en ensemble train/test, par exemple, 20% de données en test et 80% en train. Lors de la phase d'apprentissage, l'algorithme ne verra que les données de test. On comparera alors, par une erreur MSE (mean squarred error) l'erreur faite en prédisant les données de test avec les paramètres ${U}$ et ${I}$ appris par l'algorithme à l'erreur MSE faite par la baseline sur les données de test.

La baseline, ici est simple: pour chaque item, la note qu'un user lui donnera sera la note moyenne de cet item parmi les notes observées. ${\hat{R_{i,j}}^{baseline} = \frac{1}{nbUsers} \sum_{i=1}^{nbUsers} R_{i,j}^O}$



#### Cherchons un learning rate (${\alpha}$) optimal 

<img src="images/" width="300">

#### Cherchons un coefficient de régularization (${\beta}$) optimal 

<img src="images/" width="300">

#### Cherchons un nombre de facteurs optimal 

<img src="images/" width="300">

## Et maintenant, nous pouvons visualiser le résultat:

Les matrices ${U}$ et ${I}$ apprises par NMF nous donnent une information: des profils utilisateur, et des profils de série (par rapport à leur notation par les utilisateurs). Nous avons appris ${k}$ profils utilisateur et ${k}$ profils items. Dans cet espace de grande dimension (nous avons pris ${k = 125}$), chaque item (série) est représenté. Il peut donc être intéressant de visualiser cette représentation. Nous appliquons un algorithme de réduction de dimensionnalité très connu, l'algorithme PCA (analyse en composantes principales), pour pouvoir visualiser les séries en 2D en conservant le maximum d'information de la représentation en ${k}$D originale.

#insérer une petite image qui va bieng

Visualisons les embeddings de la matrice ${I}$

In [71]:
np.savetxt(f"imdb_vectors_U.tsv", learnt_U, delimiter="\t")

In [23]:
#sauvegarde des embeddings appris pour les visualiser dans tensorboard
np.savetxt(f"imdb_vectors.tsv", learnt_I.transpose(), delimiter="\t")
with open(f"imdb_metadata.tsv","w") as metadata_file:
    for x in serie_dict: #hack for space
        x = " ".join(x.split("_")[1:])
        if len(x.strip()) == 0:
            x = f"space-{len(x)}"
        metadata_file.write(f"{x}\n")

<img src="images/nmf_stargate.png" width="600" >

<img src="images/nmf_ncis.png" width="600" >


Ici, dans la visualisation en 2D issue de la PCA sur les profils Item, on remarque deux "clusters" qui se distinguent. Deux profils bien distincts semblent doncs émerger des notes.

On peut regarder plus attentivement à quoi ces deux profils correspondent: ici le cluster "droit"

<img src="images/nmf_cluster_droit.png" width="500" >

Et ici le cluster "gauche"

<img src="images/nmf_cluster_gauche.png" width="500" >

... les données n'appartenant à aucun des deux clusters, situées au "centre":

<img src="images/nmf_cluster_central.png" width="500" >


### Comment interpréter:

Deux profils se distinguent de façon bien nette. Ils correspondent à deux catégories de séries notées de façon similaire par le même "genre" d'utilisateurs (au sens des préférences). Au centre du nuage de points se trouvent des séries moins fortement liées à une catégorie d'utilisateur, qu'on pourrait qualifier de "globalement aimées" (ou détestées pour certaines?) en tout cas de séries qui ne "collent" pas fortement à un profil particulier. C'est le cas par exemple de Shameless et de Law Order, deux séries qui d'ailleurs se retrouvent très souvent recommandées ci-dessous: Ce sont des séries très bien notées (8,7 étoiles en moyenne pour Shameless) qui sont notées par des profils d'utilisateurs très variés. 

### SVD (Singular Value Decomposition) avec un biais:

**SVD** est un autre algorithme connu de factorisation de matrice. En recommandation, **il ne s'agit pas d'un vrai algorithme de SVD** mais il est néanmoins toujours appellé SVD. Il lève la contrainte de non-négativité sur ${U}$ et ${I}$ et rajoute 3 autres valeurs, ${b}$, ${bU_{i}}$, et ${bI_{j}}$.

on calcule en fait ${\hat{R_{i,j}}= \mu + \mu U_{i} + \mu I_{j} + U_{i}\cdot I_{j}}$. Voici pourquoi:

>Supposons que vous voulez une estimation de la note de Jean-Rachid sur le film Taxi 3. Supposons que la moyenne des notes sur tous les films, µ, est 6.7 étoiles. De plus, Taxi 3 a une meilleure moyenne que la moyenne des films, et il a tendance à être noté 0.5 étoiles au-dessus de la moyenne, grace au jeu d'acteur incroyable de Fred Diefenthal. De plus, Jean-Rachid est un cinéphile critique, qui a tendance à noter 0.3 étoiles en-dessous de la moyenne. Donc, une estimation de la note de Jean-Rachid sur Taxi 3 serait 6.9 étoiles (6.7 + 0.5 - 0.3).

On introduit donc une baseline, matérialisée par 3 variables, qui ne sont pas apprises par l'algorithme (donc pas modifiées).

${\mu}$ représente la moyenne globale: c'est la moyenne de **toutes** les notes, pour tous les films. ${\mu U}$ est un vecteur de taille ${m}$ qui contient pour chaque utilisateur sa déviation par rapport à la moyenne ${\mu}$ (comme dans l'exemple ci-dessus avec Jean-Rachid). ${\mu I}$ est un vecteur de taille ${n}$ qui contient pour chaque item (série, ici) sa déviation par rapport à la note moyenne. On ne reconstitue alors que la déviation par rapport à ces 3 paramètres.


Le code sous forme de classe se trouve dans <a href="https://github.com/ismaelbonneau/movie_recommender/tree/master/recommenders">le package recommenders</a>

In [ ]:
from recommenders.matrixFactorization import SVDpp
       
svdpp = SVDpp(50)

## Un peu d'hyperparameters tuning s'impose:

les paramètres à estimer sont les mêmes que pour la NMF.

In [31]:
## ici

#### Performances de nos algorithmes en test:

todo: mettre les chiffres avec algo corrigé

| **modèle**      |     5 factors   |   10 factors |   50 factors  |   100 factors   |   125 factors  |
| ------------- |: -------------: | ---------: | ---------: | ---------: | ---------: |
| **baseline**      |        0.6963        |      0.6963 |      0.6963 |      0.6963 |      0.6963 |
| **NMF**     |        0.0932        |     0.0871  |       0.08216229 |       0.0885220 |       **0.060408514** |
| **SVD++**     |        0.170291        |      0.145761 |      0.11668 |       0.1120696 |       0.104418166 |


### Revenons sur l'algorithme NMF:

Bien, nous avons vu qu'avec un paramètre ${\beta}$ de régularization et un nombre de facteurs ${k}$ bien choisis, le modèle NMF était le meilleur et arrivait même à dépasser la base line, plutôt solide.

Une étape est fondamentale dans le déroulement de l'algorithme, c'est l'étape de l'initialisation. En effet, dans le cadre de la NMF sur des matrices comportant des valeurs manquantes, la fonction à minimiser est non convexe.

<img src="images/convex_vs_nonconvex.png" width="800" >

Une fonction non convexe peut comporter plusieurs minimums locaux. Dans ce cas là, le risque pour l'algorithme est de rester coincé dans un optimum local. Il est alors important de choisir l'étape d'initialisation de l'algorithme afin de ne pas tomber dans cet optimum local. Jusqu'à présent, nous choisissions d'initialiser les matrices ${U}$ et ${I}$ avec des valeurs aléatoires tirées selon une loi normale. Pouvons-nous choisir une initialisation plus intelligente?


### Quel algorithme d'optimisation?

parler de gradient descent classique vs. les autres algos


### Vers une méthode d'initialisation plus performante:

D'après [<a href="https://fr.slideshare.net/DaichiKitamura/efficient-initialization-for-nonnegative-matrix-factorization-based-on-nonnegative-independent-component-analysis">1</a>] parler de ICA, PCA pour initialisation.

## Intégrer une mesure de similarité content-based sur les items:

Notre méthode n'intègre aucun a priori sur la similarité entre deux séries. Par exemple, elle ne prend pas en compte pour recommander à un utilisateur qui aimerait les simpsons, sa similarité avec Rick & Morty.

Nous disposons pourtant de cette information! c.f <a href="https://github.com/ismaelbonneau/movie_recommender/blob/master/similarities.ipynb">notebook similarities</a>. Nous disposons d'une mesure de similarité entre les séries. Pourquoi ne pas intégrer cette connaissance a priori à notre algorithme de filtrage collaboratif?

Pour cela, les approches sont nombreuses:

- On peut vouloir combiner les deux approches (collaborative et basée sur le contenu) a posteriori, **après les avoir calculées séparément de leur coté**. Dans ce cas, on peut d'abord penser à un simple **produit matriciel**.
- Toujours en combinant les deux a posteriori, on peut se servir de la mesure de similarité comme d'une **clé pour trier** les séries recommandées par la NMF.
- On peut aussi tenter d'**intégrer les informations basées sur le contenu dans le calcul de la factorisation matricielle**.

In [17]:
#chargement "raw" et sale pour le moment
import gensim
model = gensim.models.doc2vec.Doc2Vec.load("doc2vec/doc2vec_model1")

from utils.load_data import getMostImportantSeries
series, count = getMostImportantSeries(path)

series_toid = {}
for i in range(len(series)):
    s = series[i].replace("_s_", "s_")
    s = s.replace("__", "_")
    series_toid[s] = i
id_toserie = {series_toid[i]: i for i in series_toid.keys()}

embeddings = np.array([model.docvecs[series_toid[s]] for s in df.index])
sim = cosine_similarity(embeddings) #passage en matrice de similarités

In [18]:
print(embeddings.shape)

(171, 70)


## Intégration de mesures content-based: NMF Tri-factorization

Inspiré et adapté du travail de _Aghiles Salah_, _Melissa Ailem_, _Mohamed Nadif_, dans _Word Co-Occurrence Regularized Non-Negative
Matrix Tri-Factorization for Text Data Co-Clustering_ (<a href="https://www.aaai.org/ocs/index.php/AAAI/AAAI18/paper/view/16464/16664.pdf">lien vers l'article</a>). Merci à eux (l'article original parle de catégorisation thématique de documents).

Nous reprenons notre problème de **NMF**, avec ${R}$ la matrice ${m \times n}$ à reconstituer (approximer), ${U}$ et ${I}$ les matrices facteurs servant à approximer ${R}$.

Introduisons une matrice ${S}$, la matrice de similarité entre items de taille ${n \times n}$, où ${S_{i,j}}$ indique la similarité entre la série ${i}$ et la série ${j}$, et ${Q}$, une matrice de dimension ${k \times n}$, qui est une nouvelle matrice "facteur" comme ${U}$ et ${I}$, à apprendre. Cette matrice sera multipliée par ${I}$ durant le processus d'optimisation.

On apprend donc trois matrices ${U}$ ${I}$ et ${Q}$ (mais seules ${U}$ et ${I}$ nous seront utiles à la fin du processus). 

la fonction d'erreur est ${cost(U, I, Q)= ||R - U \cdot I||^2 + \beta (||S - I^\top \cdot Q||)}$.

Ainsi, l'algorithme apprend maintenant une matrice ${I}$ qui doit à la fois reconstruire les profils item des notes de ${R}$, mais aussi les similarités de ${S}$ la matrice de similarités. Cela permet d'intégrer la connaissance sur les similarités au processus d'optimisation et de prendre en compte les similarités entre séries dans le calcul des profils item. Il va bien sûr de soi que la MSE sera moins bonne de cette manière, puisque la similarité item ne matche pas forcément avec les profils de notes existant dans ${R}$. Néanmoins, c'est un compromis nécessaire pour arriver à produire une recommandation hybride de qualité.

In [130]:
from recommenders.matrixFactorization import triNMF #code sous forme de classe!

In [128]:
trinmf = triNMF(10)
learnt_U, learnt_I, results = trinmf.run(df, sim, train, test, 1000, 0.00001, alpha=0.001, verbose=False)

In [89]:
#sauvegarde des embeddings appris pour les visualiser dans tensorboard
np.savetxt(f"imdb_vectors_trinmf.tsv", learnt_I.transpose(), delimiter="\t")